In [ ]:
from helper_load import get_engine_database
from collections import defaultdict
from random import shuffle
import pandas as pd
import re

with open("./secrets.txt", "r") as f:
    driver, username, password, host, port, database = f.read().split(",")
engine = get_engine_database(driver, username, password, host, port, database)

In [ ]:
socios = pd.read_excel("Empresas-Sócios_CNAE-9602501_Cidade-Maringa_test_1.xlsx", sheet_name="Sócios")
socios.head()

In [ ]:
def get_data(nomes: list[str], cnpjs_cpfs: list[str], telefones_coletados: list[str]=[]) -> tuple[dict, set]:
    socio_numero_dict = defaultdict(str)
    for nome, cnpj_cpf in zip(nomes, cnpjs_cpfs):
        with engine.begin() as conn:
            qry = f"""SELECT telefone FROM public.socio_telefone
            WHERE nome_socio = '{nome}' AND cnpj_cpf_socio = '{cnpj_cpf}' AND telefone <> ' ';"""

            socio_numero_dict[f"{nome}-{cnpj_cpf}"] = pd.read_sql(qry, conn).dropna().drop_duplicates()["telefone"].values[0]

    todos_telefones = set()
    for _, t in dict(socio_numero_dict).items():
        todos_telefones.add(t)

    if telefones_coletados:
        return dict(socio_numero_dict), set([t for t in todos_telefones if t not in telefones_coletados])
    
    return dict(socio_numero_dict), todos_telefones

def get_socios_telefones(nomes: list[str],
                         cnpjs_cpfs: list[str],
                         socio_numero_dict: dict[str],
                         todos_telefones: set,
                         socios_telefones: list[tuple] = []) -> tuple[list[tuple], list[tuple]]:
    socios_telefones_replace = []
    for nome, cnpj_cpf in zip(nomes, cnpjs_cpfs):
        ts = socio_numero_dict[f"{nome}-{cnpj_cpf}"]
        if ts in todos_telefones:
            socios_telefones.append((nome, cnpj_cpf, ts))
            todos_telefones.remove(ts)
        else:
            socios_telefones_replace.append((nome, cnpj_cpf))
    return socios_telefones, socios_telefones_replace

In [ ]:
nomes, cnpjs_cpfs = socios["Nome Sócio"], socios["CNPJ/CPF Sócio"]

socio_numero_dict, todos_telefones = get_data(nomes, cnpjs_cpfs)
socios_telefones, socios_telefones_replace = get_socios_telefones(nomes, cnpjs_cpfs, socio_numero_dict, todos_telefones)

In [ ]:
count = 0
while True:
    nomes, cnpjs_cpfs = [i[0] for i in socios_telefones_replace], [i[1] for i in socios_telefones_replace]

    socio_numero_dict, todos_telefones = get_data(nomes, cnpjs_cpfs, [i[-1] for i in socios_telefones])
    novo_socios_telefones, socios_telefones_replace = get_socios_telefones(nomes, cnpjs_cpfs, socio_numero_dict, todos_telefones, socios_telefones)

    if len(socios_telefones) == len(novo_socios_telefones):
        socios_telefones = novo_socios_telefones
        print(count)
        break
    count += 1

    socios_telefones = novo_socios_telefones

In [ ]:
socios_telefones_replace

In [ ]:
with engine.begin() as conn:
    qry = f"""SELECT DISTINCT
                CASE
                    WHEN telefone1 IS NOT NULL THEN CONCAT(ddd1,' ',telefone1)
                    ELSE CONCAT(ddd2,' ',telefone2)
                END AS telefone
            FROM public.estabelecimentos
            WHERE
                ((ddd1 IS NOT NULL AND telefone1 IS NOT NULL) OR
                (ddd2 IS NOT NULL AND telefone2 IS NOT NULL)) AND
                situacao_cadastral <> 2;"""
    rand_telefones_chunk = pd.read_sql(qry, conn, chunksize=100_000)

r = []
for chunk in rand_telefones_chunk:
    for value in chunk["telefone"].unique():
        v = value.strip()
        if re.match(r"[0-9]{2,} [0-9]+", v):
            r.append(v)
    break

shuffle(r)
rand_telefones = r[:len(socios_telefones_replace)]

for nc, t in zip(socios_telefones_replace, rand_telefones):
    socios_telefones.append((nc[0], nc[1], t))

In [ ]:
st = pd.DataFrame(data=socios_telefones, columns=["Nome Sócio", "CNPJ/CPF Sócio", "Telefone"])

In [65]:
socios_final = pd.merge(socios, st, how="inner", on=["Nome Sócio","CNPJ/CPF Sócio"])